In [20]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from collections import deque
import random
import time
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Dropout, BatchNormalization, LSTM
from keras.callbacks import TensorBoard, ModelCheckpoint
import tensorflow as tf
from keras.optimizers import Adam

In [30]:
SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 3
RATIO_TO_PREDICT = 'bitcoin'
EPOCHS = 100
BATCH_SIZE = 64
NAME = RATIO_TO_PREDICT + '-' + str(SEQ_LEN) + "-SEQ-" + str(FUTURE_PERIOD_PREDICT) + "-PRED-" + str(int(time.time()))

In [22]:
main_df = pd.DataFrame()

ratios = ['stratis', 'monero', 'waves', 'bitcoin', 'litecoin',
          'numeraire', 'dash', 'omisego', 'bitcoin_cash', 'ethereum_classic',
          'ripple', 'ethereum', 'bitconnect', 'nem', 'iota', 'neo',
          'qtum']

In [23]:
for ratio in ratios:
    dataset = "dataset/" + ratio + '_price' + ".csv"
    
    df = pd.read_csv(dataset)
    
    df.rename(columns={'Close': ratio + '_Close', 'Volume': ratio + '_Volume'}, inplace=True)
    
    df.set_index('Date', inplace=True)
    df = df[[ratio + '_Close', ratio + '_Volume']]
    
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)

In [24]:
# main_df = pd.DataFrame() # begin empty

# ratios = ["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"] 

In [25]:
# for ratio in ratios:
#     dataset = "dataset2/" + ratio + ".csv"
    
#     df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'Close', 'Volume'])
    
#     df.rename(columns={'Close': ratio + '_Close', 'Volume': ratio + '_Volume'}, inplace=True)
    
#     df.set_index('time', inplace=True)
#     df = df[[ratio + '_Close', ratio + '_Volume']]
    
#     if len(main_df) == 0:
#         main_df = df
#     else:
#         main_df = main_df.join(df)

In [26]:
print(main_df.head())

           stratis_Close  stratis_Volume  monero_Close  monero_Volume  \
Date                                                                    
20-Feb-18           9.13        21055000        304.40       71114700   
19-Feb-18           9.95        22415800        316.49       67482400   
18-Feb-18           9.26        14139400        300.12       68135100   
17-Feb-18           9.76        19415600        325.67      134269000   
16-Feb-18           9.56        14430300        296.48       64320200   

           waves_Close  waves_Volume  bitcoin_Close bitcoin_Volume  \
Date                                                                 
20-Feb-18         8.03      46087500        11403.7     1336605408   
19-Feb-18         8.10      37813100        11225.3     -937844592   
18-Feb-18         7.54      40081300        10551.8      154075408   
17-Feb-18         7.84      34458200        11112.7       70945408   
16-Feb-18         7.90      44143700        10233.9    -1293774592  

In [27]:
print(main_df.columns)

Index(['stratis_Close', 'stratis_Volume', 'monero_Close', 'monero_Volume',
       'waves_Close', 'waves_Volume', 'bitcoin_Close', 'bitcoin_Volume',
       'litecoin_Close', 'litecoin_Volume', 'numeraire_Close',
       'numeraire_Volume', 'dash_Close', 'dash_Volume', 'omisego_Close',
       'omisego_Volume', 'bitcoin_cash_Close', 'bitcoin_cash_Volume',
       'ethereum_classic_Close', 'ethereum_classic_Volume', 'ripple_Close',
       'ripple_Volume', 'ethereum_Close', 'ethereum_Volume',
       'bitconnect_Close', 'bitconnect_Volume', 'nem_Close', 'nem_Volume',
       'iota_Close', 'iota_Volume', 'neo_Close', 'neo_Volume', 'qtum_Close',
       'qtum_Volume'],
      dtype='object')


In [28]:
def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

In [31]:
main_df['future'] = main_df[RATIO_TO_PREDICT + '_Close'].shift(- FUTURE_PERIOD_PREDICT)
print(main_df.head())

           stratis_Close  stratis_Volume  monero_Close  monero_Volume  \
Date                                                                    
20-Feb-18           9.13        21055000        304.40       71114700   
19-Feb-18           9.95        22415800        316.49       67482400   
18-Feb-18           9.26        14139400        300.12       68135100   
17-Feb-18           9.76        19415600        325.67      134269000   
16-Feb-18           9.56        14430300        296.48       64320200   

           waves_Close  waves_Volume  bitcoin_Close bitcoin_Volume  \
Date                                                                 
20-Feb-18         8.03      46087500        11403.7     1336605408   
19-Feb-18         8.10      37813100        11225.3     -937844592   
18-Feb-18         7.54      40081300        10551.8      154075408   
17-Feb-18         7.84      34458200        11112.7       70945408   
16-Feb-18         7.90      44143700        10233.9    -1293774592  

In [11]:
print(main_df[[RATIO_TO_PREDICT + '_Close', 'future']].head())

            ETH-USD_Close     future
time                                
1528968660            NaN  485.75000
1528968720      486.01001  486.00000
1528968780      486.00000  486.00000
1528968840      485.75000  485.98999
1528968900      486.00000  485.98999


In [32]:
main_df['target'] = list(map(classify, main_df[RATIO_TO_PREDICT + '_Close'], main_df['future']))

In [33]:
print(main_df[[RATIO_TO_PREDICT + '_Close', 'future', 'target']].head(20))

           bitcoin_Close    future  target
Date                                      
20-Feb-18       11403.70  11112.70       0
19-Feb-18       11225.30  10233.90       0
18-Feb-18       10551.80  10166.40       0
17-Feb-18       11112.70   9494.63       0
16-Feb-18       10233.90   8598.31       0
15-Feb-18       10166.40   8926.57       0
14-Feb-18        9494.63   8129.97       0
13-Feb-18        8598.31   8621.90       1
12-Feb-18        8926.57   8736.98       0
11-Feb-18        8129.97   8265.59       1
10-Feb-18        8621.90   7621.30       0
09-Feb-18        8736.98   7754.00       0
08-Feb-18        8265.59   6955.27       0
07-Feb-18        7621.30   8277.01       1
06-Feb-18        7754.00   9174.91       1
05-Feb-18        6955.27   8830.75       1
04-Feb-18        8277.01   9170.54       1
03-Feb-18        9174.91  10221.10       1
02-Feb-18        8830.75  10106.30       1
01-Feb-18        9170.54  11296.40       1


In [34]:
times = sorted(main_df.index.values)
last_5_pct = times[-int(0.05 * len(times))]

In [35]:
validation_main_df = main_df[(main_df.index >= last_5_pct)]
main_df = main_df[(main_df.index < last_5_pct)]

In [36]:
def preprocess_df(df):
    df = df.drop('future', 1)
    
    for col in df.columns:
        if col != 'target':
            df[col] = df[col].replace(',','').astype(float).pct_change()
            df.dropna(inplace = True)
            
            df[col] = preprocessing.scale(df[col].values)
            
    df.dropna(inplace = True)
    
    sequential_data = []
    prev_days = deque(maxlen = SEQ_LEN)
    
    for i in df.values:
        prev_days.append([n for n in i[:-1]])
        if len(prev_days) == SEQ_LEN:
            sequential_data.append([np.array(prev_days), i[-1]])
    
    random.shuffle(sequential_data)
    
    buys = []
    sells = []
    
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq, target])
        elif target == 1:
            buys.append([seq, target])
            
    random.shuffle(buys)
    random.shuffle(sells)
    
    lower = min(len(buys), len(sells))
    
    buys = buys[:lower]
    sells = sells[:lower]
    
    sequential_data = buys + sells
    
    random.shuffle(sequential_data)
    
    X = []
    y = []
    
    for seq, target in sequential_data:
        X.append(seq)
        y.append(target)
        
    return np.array(X), y

In [37]:
train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(main_df)

In [38]:
print("Train Data: " + str(len(train_x)), "Validation Data: " + str(len(validation_x)))
print("Don't Buys: " + str(train_y.count(0)), "Buys: " + str(validation_y.count(1)))
print("Validation Don't Buys: " + str(validation_y.count(0)), "Buys: " + str(validation_y.count(1)))

Train Data: 86 Validation Data: 86
Don't Buys: 43 Buys: 43
Validation Don't Buys: 43 Buys: 43


In [ ]:
model = Sequential()

model.add(CuDNNLSTM(128, input_shape = (train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(CuDNNLSTM(128, input_shape = (train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(CuDNNLSTM(128, input_shape = (train_x.shape[1:])))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

opt = Adam(lr=0.001, decay=1e-6)

model.compile(loss='sparse_categorical_crossentropy',
              optimizer = opt,
              metrics = ['accuracy'])

tensorboard = TensorBoard(log_dir = 'logs/' + NAME)

filepath = 'RNN_Final-{epoch:02d}-{val_acc:.3f}'
checkpoint = ModelCheckpoint('models/{}.model'.format(filepath, monitor='val_acc',
                                                      verbose=1,
                                                      save_best_only = True,
                                                      mode = 'max'))

history =  model.fit(train_x, train_y,
                     batch_size = BATCH_SIZE,
                     epochs = EPOCHS,
                     validation_data = (validation_x, validation_y),
                     callbacks = [tensorboard, checkpoint])